## Automatic Summarization

Lo scopo, è di creare automaticamente un riassunto dei documenti proposti in `./data`, e valutarne i risultati (BLEU o ROUGE)

### Algoritmo semplice
1. Individuare il topic del testo da riassumere (può essere individuato tramite vettori NASARI)
2. Crea il contesto raccogliendo i vettori dei termini (si può ripetere ma non c'è uno stopping criterion specificato. Potrebbe essere fatto solo per il primo paragrafo)
3. Ritieni i paragrafi in cui le frasi contengono i termini più salienti, basandosi sul WeightedOverlap

### Problemi da affrontare
* Content Selection: Quale informazione selezionare, e a che granularità? (si assumono frasi)
* \[Opzionale\]: Come riordinare e strutturare l'informazione estratta?
* \[Opzionale\]: Come si ri-pulisce il testo per migliorarne la coerenza?

Se è necessario, scaricare dd-nasari.txt.gz (600Mb) con il seguente snippet:

In [ ]:
#import wget
#file = wget.download("http://www.di.unito.it/~radicion/tmp2del/TLN_180430/dd-nasari.txt.tgz")

In [2]:
import nltk
import math
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from common import utils as utils
from IPython.display import display
from common import prettyprint as pp

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

debug = True
def debug_trace(phrase):
    if debug:
        print(phrase)
        print()

In [3]:
# Sumy imports
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

Prima di tutto, parsifichiamo i contenuti del sottoinsieme di nasari

In [4]:
nasari_exp = open('./data/dd-nasari.txt', 'r').read()
nasari = [tuple(line.split(';')[1:]) for line in nasari_exp.split('\n')]
nasari = dict([(i[0], list(i[1:])) for i in nasari if i])

La seguente funzione restituisce il 'contesto' (una lista di vettori nasari), partendo da una lista di parole in input (può prendere anche solo una parola in input). Il parametro `include_weights`, specifica se includere o meno i pesi delle dimensioni di nasari (falso di default)

In [5]:
def context_from_words(wordlist, include_weights=False):
    if type(wordlist) == str:
        wordlist = [wordlist]
    if include_weights:
        return utils.flatten(
            [nasari[word.capitalize()] for word in wordlist if word.capitalize() in nasari.keys()]
        )
    else:
        return utils.flatten(
            [map(lambda i: i.split('_')[0], nasari[word.capitalize()]) for word in wordlist if word.capitalize() in nasari.keys()]
        )

La seguente funzione, prende in input dei vettori nasari ed un limite alla ricerca di ulteriore contesto. Procede a ricorrere su tutti i termini dei vettori, catturandoli via via per creare il contesto `limit` volte.

In [6]:
def get_recursive_context(starting_vectors, limit=2):
    vectors = starting_vectors
    result = []
    wordset = set(vectors)

    # terms with a lesser rank are _already_ less valuable.
    max_weight = len(nasari[list(nasari.keys())[0]])

    # since the parsed nasari it's a dict {term: [terms]};
    # here we have only [terms]
    for iteration in range(1, limit):
        for dimension in vectors:
            if dimension != []:
                # Since dimensions are sorted already, start from the maximum importance (the total dimensions)
                # Subtract the vector's position (plus one to account for the start at 0)
                # Modulus max_weight since they aren't separated in any way.
                result.append((dimension, (max_weight-(vectors.index(dimension) % max_weight)+1)/iteration))
        new_items = utils.flatten([context_from_words(dim) for dim in vectors])
        vectors.clear()
        vectors += new_items
    temp = sorted(list(set(result)), key=lambda i: i[1], reverse=True)
    
    clean_dict = {}
    # Remove dupes with lesser score
    for i in temp:
        if i[0] not in clean_dict.keys():
            clean_dict[i[0]] = i[1]
    return clean_dict

La seguente funzione parte da una stringa e ne restituisce una versione senza punteggiatura e lemmatizzata, in forma di stringa.

In [7]:
def get_lemmas(text):
    punctuation = list(string.punctuation)
    punctuation += ['’', '–', '‘', '“', '”', '©']
    tokens = nltk.word_tokenize(text)
    return [nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) for token in tokens if (token.lower() not in punctuation)]

In [8]:
# Piccola funzione per prendere direttamente il tfidf
def get_tfidf(word, phrase_idx, matrix):
    return matrix.iloc[phrase_idx][word]

Per dare una stima corretta in fase di valutazione è essenziale che gli articoli che vengono riassunti siano presi dagli stessi dati. In fase di valutazione uso direttamente il parser di sumy per ottenere un riassunto 'gold standard'. Dal momento che l'articolo potrebbe essere stato aggiornato e/o contenere ulteriori frasi non presenti nel testo proposto, sfrutto lo stesso parser solo per ottenere i dati di partenza, in modo da non ottenere differenze dovute a questo in fase di valutazione.

In [9]:
def get_url_from_article(path):
    f = open(path, 'r')
    contents = f.read()
    f.close
    return contents.split('\n')[0].replace('# ', '')

def contents_from_url(url):
    result = []
    parser = HtmlParser.from_url(url, Tokenizer("english"))
    for paragraph in parser.document.paragraphs:
        result.append([str(sent) for sent in paragraph.sentences])
        #for sentence in paragraph.sentences:
            #print(sentence)
    return result

Anzitutto, per processare l'articolo, lo suddivido in paragrafi, che saranno a loro volta suddivisi in frasi dal `sent_tokenizer` di nltk.

In [10]:
article_data = open('./data/Andy-Warhol.txt').read()
# Use nltk to tokenize phrases from the article
article_paragraphs = article_data.split('\n\n')
article_phrases = [nltk.sent_tokenize(paragraph) for paragraph in article_paragraphs]
phraselist = utils.flatten(article_phrases)

Calcolo il Tf.Idf per ogni parola, per ogni frase, l'ho preferito perché è una statistica più robusta della frequenza.

In [11]:
vectorizer = TfidfVectorizer(tokenizer=get_lemmas, stop_words=stopwords.words('english'))
vectors = vectorizer.fit_transform(utils.flatten(article_phrases))
tfidfm = pd.DataFrame(vectors.todense().tolist(), columns=vectorizer.get_feature_names())

### Position-based
Uso la prima euristica proposta e prendo la prima frase nel paragrafo, per ogni paragrafo (**indipendentemente dal suo score**). Per quanto riguarda l'ultima frase, la si valuta come tutte le altre, prima di inserirla (7% non è una buona chance)


#### Topic Signature
Converto il topic in un modello bag-of-words, rimuovo le parole 'irrilevanti', quello che rimane è la 'topic signature'.

Per capire quali sono le parole rilevanti, calcolo la media delle frequenze, e la uso come soglia per le stop-words (questo perché osservando la distribuzione delle parole nei testi proposti, la maggioranza delle parole sono piuttosto irrilevanti (espresse solo una volta)).

In [25]:
topic_freqs = utils.word_frequencies(
    list(filter(lambda i: i not in stopwords.words('english'),
        utils.flatten([get_lemmas(phrase) for phrase in utils.flatten(article_phrases)])
    ))
)
stopword_treshold = math.ceil(np.mean(list(zip(*topic_freqs))[1]))
debug_trace(f"{pp.info('Removing words less frequent than: ')}: {stopword_treshold}")
topic_signature = list(filter(lambda i: i[1] > stopword_treshold, topic_freqs))
debug_trace(f"{pp.info('Topic Signature: ')}: {topic_signature}")

#### Contesto
Per creare il contesto sfrutto le parole della topic signature. Dopo aver fatto la tokenizzazione della frase (`utils.preprocess_phrase_nostem`), ripulisco il testo di queste prime frasi da eventuali segni di punteggiatura (per evitare che finiscano nel contesto). Quindi uso NASARI per cercare tutte le parole correlate a quella proposta, a causa delle dimensioni osservate, questo viene ripetuto solo due volte.

In [26]:
keywords = context_from_words(list(zip(*topic_signature))[0])
context = get_recursive_context(keywords)
debug_trace(f"{pp.info('Augmented Context')}: {context}")

Nel contesto estratto, ci sono alcune stopwords, quindi le si filtra.

In [28]:
topic_words = list(zip(*topic_signature))[0]
context_words = [context_word for context_word in list(context.keys()) if context_word not in stopwords.words('english')]

La prossima funzione definisce una misura di salienza. L'idea è di sommare i tf/idf di ogni parola nell'overlap lessicale. Dando punti bonus (`len(topic_overlap)`) per le frasi che contengono parole della topic signature, e per le parole contestuali (`len(context_overlap) * .5`) ma riducendone il peso di metà (dal momento che sono prese da nasari invece che direttamente dal testo)

In [15]:
def saliency(phrase, phrase_list, tfidfm, topic_words, context_words):
    phrase_lemmas = get_lemmas(phrase)
    phrase_idx = phrase_list.index(phrase)
    topic_overlap = list(set(phrase_lemmas).intersection(set(topic_words)))
    topic_score = sum([get_tfidf(word, phrase_idx, tfidfm) for word in topic_overlap])
    context_overlap = list(set(phrase_lemmas).intersection(set(context_words)))
    context_score = sum([get_tfidf(word, phrase_idx, tfidfm) for word in context_overlap])
    return topic_score + len(topic_overlap) + context_score + len(context_overlap) * .5


Sfruttando la misura appena definita, ho mappato le frasi al loro score e riordinate per salienza decrescente.

In [30]:
saliency_scores = list(map(lambda phrase: saliency(phrase,phraselist, tfidfm, topic_words, context_words), phraselist))
weighted_phrases = sorted(list(zip(phraselist, saliency_scores)), key=lambda i: i[1], reverse=True)

Prendo come soglia del valore di salienza accettabile, il peso dell'ultimo paragrafo utile prima di sforare la soglia di compressione.

In [33]:
compression = .3
new_len = math.floor(len(phraselist) * (1-compression))
phrase_treshold = weighted_phrases[new_len-1][1]

result = []
for paragraph in article_phrases:
    for phrase in paragraph:
        # Heuristic #1: If it's the first phrase of the paragraph, take it.
        if paragraph.index(phrase) == 0:
            result.append(phrase)
        # Else, check its saliency
        elif saliency(phrase, phraselist, tfidfm, topic_words, context_words) > phrase_treshold:
            result.append(phrase)
debug_trace(result)

['# https://www.independent.co.uk/arts-entertainment/art/features/andy-warhol-tate-modern-pop-art-elton-john-marilyn-monroe-elvis-presley-a9375021.html', '\nAndy Warhol: Why the great Pop artist thought ‘Trump is sort of cheap’', 'He anticipated celebrity culture and social media, thought artists should do more than just hold a paintbrush, and wound up John Lennon.', 'As a new Tate exhibition opens, Alastair Smart shows how far the most important artist of the modern age was ahead of his time.', 'uring last year’s Super Bowl, 100 million US viewers were treated to a most unexpected sight in one of the commercial breaks.', 'It was Andy Warhol doing nothing more than taking bites out of a Burger King Whopper – and adding the occasional bit of ketchup – for 45 seconds.', 'There was no music, no punchline, just a little, light rustling of the burger’s wrapper – in a slowly unfolding scene that culminated with the hashtag #EatLikeAndy.', 'It was about as far removed as one could imagine fro

La prossima funzione riassume quanto fatto fin'ora, solo mettendo tutto assieme per restituire una lista di stringhe in output

In [34]:
def summarize(filename, compression=.3):
    # get the articles and parse them
    contents = contents_from_url(get_url_from_article(filename))
    # convert [[phrase]] -> [phrase]
    phraselist = utils.flatten(contents)
    # Compute tf.idf matrix
    vectorizer = TfidfVectorizer(tokenizer=get_lemmas, stop_words=stopwords.words('english'))
    vectors = vectorizer.fit_transform(phraselist)
    tfidfm = pd.DataFrame(vectors.todense().tolist(), columns=vectorizer.get_feature_names())
    # Compute topic signature
    topic_freqs = utils.word_frequencies(
    list(filter(lambda i: i not in stopwords.words('english'),
        utils.flatten([get_lemmas(phrase) for phrase in phraselist])
        ))
    )
    stopword_treshold = math.ceil(np.mean(list(zip(*topic_freqs))[1]))
    debug_trace(f"{pp.info('Removing words less frequent than: ')}: {stopword_treshold}")
    topic_signature = list(filter(lambda i: i[1] > stopword_treshold, topic_freqs))
    debug_trace(f"{pp.info('Topic Signature: ')}: {topic_signature}")
    # Compute Augmented Context
    keywords = context_from_words(list(zip(*topic_signature))[0])
    context = get_recursive_context(keywords)
    topic_words = list(zip(*topic_signature))[0]
    context_words = [context_word for context_word in list(context.keys()) if context_word not in stopwords.words('english')]
    debug_trace(f"{pp.info('Augmented Context: ')}: {context}")
    saliency_scores = list(map(lambda phrase: saliency(phrase, phraselist, tfidfm, topic_words, context_words), phraselist))
    weighted_phrases = sorted(list(zip(phraselist, saliency_scores)), key=lambda i: i[1], reverse=True)
    new_len = math.floor(len(phraselist) * (1-compression))
    phrase_treshold = weighted_phrases[new_len-1][1]
    result = []
    for paragraph in contents:
        for phrase in paragraph:
            # Heuristic #1: If it's the first phrase of the paragraph, take it.
            if paragraph.index(phrase) == 0:
                result.append(phrase)
            # Else, check its saliency
            elif saliency(phrase, phraselist, tfidfm, topic_words, context_words) > phrase_treshold:
                result.append(phrase)
    return result



## Valutazione

In [35]:
def get_gold_summary(filename, compression=.3):
    url = open(filename).read().split('\n')[0].replace('# ', '')
    debug_trace(f"{pp.info('Extracted URL')}: {url}")
    LANGUAGE = "english"
    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    SENTENCES_COUNT = math.floor(len(parser.document.sentences) * (1-compression))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    return [str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT)]

In [36]:
def rouge(summary, gold_standard):
    return len(set(summary).intersection(set(gold_standard)))/len(summary)

In [39]:
def round_evaluate(files):
    results = dict()
    for compression in [.1, .2, .3, .4, .5, .6, .7, .8, .9]:
        for article in files:
            original = summarize(article, compression=compression)
            gold = get_gold_summary(article, compression=compression)
            score = rouge(original, gold)
            results[f"{article}@{compression}"] = score
            if score > .5:
                print(f"{pp.info(article)}@{pp.info(compression)}: {pp.success(score)}")
            else:
                print(f"{pp.info(article)}@{pp.info(compression)}: {pp.fail(score)}")
        print()
    return results
            


## Risultati

In [40]:
debug=False
results = round_evaluate(['./data/Andy-Warhol.txt', './data/Ebola-virus-disease.txt', './data/Life-indoors.txt', './data/Napoleon-wiki.txt', './data/Trump-wall.txt'])

./data/Andy-Warhol.txt@0.1: 0.86
./data/Ebola-virus-disease.txt@0.1: 0.8276062900407688
./data/Life-indoors.txt@0.1: 0.9615384615384616
./data/Napoleon-wiki.txt@0.1: 0.933873986275733
./data/Trump-wall.txt@0.1: 0.7564526803441429

./data/Andy-Warhol.txt@0.2: 0.782608695652174
./data/Ebola-virus-disease.txt@0.2: 0.7616822429906542
./data/Life-indoors.txt@0.2: 0.8461538461538461
./data/Napoleon-wiki.txt@0.2: 0.8864864864864865
./data/Trump-wall.txt@0.2: 0.7605947955390334

./data/Andy-Warhol.txt@0.3: 0.7142857142857143
./data/Ebola-virus-disease.txt@0.3: 0.7649700598802395
./data/Life-indoors.txt@0.3: 0.782608695652174
./data/Napoleon-wiki.txt@0.3: 0.8806774441878368
./data/Trump-wall.txt@0.3: 0.7249575551782682

./data/Andy-Warhol.txt@0.4: 0.5789473684210527
./data/Ebola-virus-disease.txt@0.4: 0.7439236111111112
./data/Life-indoors.txt@0.4: 0.6666666666666666
./data/Napoleon-wiki.txt@0.4: 0.8521816562778273
./data/Trump-wall.txt@0.4: 0.7118811881188118

./data/Andy-Warhol.txt@0.5: 0.514

Bisogna arrivare a comprimere il testo del 90% per avere performance in recall molto basse su tutti gli articoli. Bisogna comprimere il testo al 60% per avere una performance molto bassa su 2/5 articoli. Gli articoli sono lunghi (in parole), rispettivamente:

`./data/Andy-Warhol.txt`: 1227  
`./data/Ebola-virus-disease.txt`: 2106  
`./data/Life-indoors.txt`: 567  
`./data/Napoleon-wiki.txt`: 1006  
`./data/Trump-wall.txt`: 3738  

I primi a perdere sufficientemente in recall sono Andy-Warhol e Life-indoors. All'inizio ho pensato che avesse a che fare con la lunghezza originale dell'articolo, se non fosse che l'articolo su Andy Warhol è il doppio più lungo di Life-indoors e il terzo per lunghezza.

L'ipotesi che reputo più probabile è che il modo che ho intrapreso per fare il riassunto (l'euristica 'position-based' e i vettori di nasari) è una buon modo per riassumere pagine di Wikipedia. Che ha senso, anche perché nasari è creata a partire da concetti su BabelNet (creata a partire da WordNet e Wikipedia). Mentre invece la performance peggiora molto e molto prima, su articoli di natura generale.